## <strong> 12. 결측 데이터 처리

In [1]:
# 필요한 라이브러리
import pandas as pd
import numpy as np

#### <strong> 결측치의 표현: None 객체

In [3]:
# None 객체를 포함한 배열
arr1 = np.array([1, 2, None, 4])
print(arr1.dtype)

# 결측치가 없는 배열
arr2 = np.array([1, 2, 3, 4])
print(arr2.dtype)#파이썬에서 배열에서 여러개의 데이터 타입이 섞이면 기본적으로 object로 할당됨

object
int32


In [5]:
# Object 객체는 numpy, pandas 연산과 호환되지 않음
arr1.sum() #Object과 정수간의 수치연산이 불가능 하기 떄문에 오류남

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### <strong> 결측치의 표현: NaN 값

In [9]:
# NaN 값을 포함한 배열 객체
arr3 = np.array([1, np.nan, 3, 4])#np.nan을 사용하면 숫자의 결측치로 인식됨
arr3.dtype#따라서 float64로 인식됨

dtype('float64')

In [10]:
# NaN 값에 대한 산술 연산
print(1 + np.nan)
print(0 * np.nan)
print(arr3.sum())

nan
nan
nan


In [11]:
# NaN값을 무시하는 집계함수(numpy)
print(np.nansum(arr3))
print(np.nanmax(arr3))
print(np.nanmin(arr3))

8.0
4.0
1.0


In [12]:
# Pandas는 None 객체를 자동으로 NaN값으로 변환한다.

ser = pd.Series([1, np.nan, 2, None])
print(ser)

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64


### <strong> Pandas를 이용한 결측치 처리
---

#### <strong>마스킹을 통한 Null 값 표현
+ ```isnull()```전체 결측 데이터에 대한 부울 배열(마스크) 생성
+ ```notnull()```isnll()의 역연산
* ```dropna()```null값 제거

In [13]:
# [+] 결측치를 포함한 Series 객체 생성
ser = pd.Series([1, np.nan, 2, None])

In [15]:
# [+] isnull()
ser.isnull()#결측치가 있는지 확인하는 메서드

0    False
1     True
2    False
3     True
dtype: bool

In [17]:
# [+] notnull()
ser.notnull()#결측치가 아닌지 확인하는 메서드

0     True
1    False
2     True
3    False
dtype: bool

#### <strong> 결측치를 포함한 행 또는 열 삭제
+ dropna()

In [18]:
# DataFrame 생성 함수
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind]
           for c in cols}

    return pd.DataFrame(data, ind)

df = make_df('ABC', [0, 1, 2])
df

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [20]:
# [+] NaN 값 추가
df.iloc[0,1] = np.nan
df.iloc[2,0] = np.nan
df

,A,B,C
0,A0,NaN,C0
1,A1,B1,C1
2,NaN,B2,C2


In [ ]:
# [+] 결측치를 포함한 행 삭제
df.dropna(axis=0)#axis=0은 행을 의미함


,A,B,C
1,A1,B1,C1


In [23]:
# [+] 결측치를 포함한 열 삭제
df.dropna(axis=1)#axis=1은 열을 의미함

,C
0,C0
1,C1
2,C2


In [24]:
# [+] NaN 값 추가
df.iloc[:,2]=np.nan
df

,A,B,C
0,A0,NaN,NaN
1,A1,B1,NaN
2,NaN,B2,NaN


In [25]:
# [+] NaN값이 한 개라도 포함되면 행 또는 열 삭제
df.dropna(how='any', axis=0)#how='any'는 하나라도 결측치가 있으면 삭제

,A,B,C


In [29]:
# [+] 모든 값이 전부 NaN값인 행 또는 열 삭제
df.dropna(how='all', axis=0)

,A,B,C
0,A0,NaN,NaN
1,A1,B1,NaN
2,NaN,B2,NaN


In [33]:
# [+] Non-null(정상으로 측정된) 값의 개수가 임계치보다 작은 행을 삭제->thresh 매개변수 사용
df.dropna(axis=0, thresh=2)#thresh=2는 2개 이상의 값이 있어야 삭제하지 않음

,A,B,C
1,A1,B1,NaN


In [34]:
# [+] Non-null 값의 개수가 임계치보다 작은 열을 삭제
df.dropna(axis=1, thresh=2)#thresh=2는 2개 이상의 값이 있어야 삭제하지 않음

,A,B
0,A0,NaN
1,A1,B1
2,NaN,B2


#### <strong> NaN값을 다른 값을 대체
+ ```fillna()```

In [35]:
# Series 객체 생성
ser = pd.Series([1, np.nan, 2, None, 3],
                index=list('abcde'))
ser

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [36]:
# [+] 결측치를 특정 값을 대체
df.fillna(0)#결측치를 0으로 대체

,A,B,C
0,A0,0,0
1,A1,B1,0
2,0,B2,0


In [40]:
ser.interpolate()#결측치 처리를 선형방식으로 하는 경우

a    1.0
b    1.5
c    2.0
d    2.5
e    3.0
dtype: float64

In [ ]:
# [+] Forward-fill 방식(= LOCF) 결측 구간 직전에 측정된 값으로 대체
df.fillna(method='ffill')#결측치를 직전 값으로 대체, 근데 이렇게 쓰면 경고문이 하나 생김(구식버전이라고)

C:\Users\yun\AppData\Local\Temp\ipykernel_2800\1420940570.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill')#결측치를 직전 값으로 대체


,A,B,C
0,A0,NaN,NaN
1,A1,B1,NaN
2,A1,B2,NaN


In [41]:
# [+] Backward-fill 방식(= NOCB)결측 구간 직후에 측정된 값으로 대체
df.fillna(method='bfill')#결측치를 직후 값으로 대체, 근데 이렇게 쓰면 경고문이 하나 생김(구식버전이라고)

C:\Users\yun\AppData\Local\Temp\ipykernel_2800\1694699620.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill')#결측치를 직후 값으로 대체, 근데 이렇게 쓰면 경고문이 하나 생김(구식버전이라고)


,A,B,C
0,A0,B1,NaN
1,A1,B1,NaN
2,NaN,B2,NaN


### <strong> 결측치 처리 예제: Air Quality 데이터셋

이탈리아 공기질에 대한 데이터셋임

In [42]:
from datetime import datetime
from matplotlib import pyplot as plt

"""
Load AirQualityUCI Data
"""

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

FileNotFoundError: [Errno 2] No such file or directory: './data/AirQualityUCI_refined.csv'

In [43]:
# 데이터 요약
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       2 non-null      object
 1   B       2 non-null      object
 2   C       0 non-null      object
dtypes: object(3)
memory usage: 96.0+ bytes


In [ ]:
# 시각화 설정 옵션
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 13
plt.ion()

### missingno
+ 데이터셋에서 누락된 데이터를 시각적으로 파악하고 분석하는 데 도움을 주는 Python 라이브러리입니다. 주로 데이터셋의 결측치를 시각화하여 어떤 부분이 비어 있는지, 데이터의 패턴을 파악하는 데 사용됩니다

In [44]:
# 결측 데이터 시각화
import missingno

missingno.matrix(df)

ModuleNotFoundError: No module named 'missingno'

In [ ]:
# 일산화탄소(CO) 시계열 시각화
df['CO(GT)'].plot()

In [ ]:
# 결측치 대치
imp_locf = df['CO(GT)'].copy().ffill() # LOCF
imp_nocb = df['CO(GT)'].copy().bfill() # NOCB
imp_linear = df['CO(GT)'].copy().interpolate() # 선형 보간
imp_mean = df['CO(GT)'].copy().fillna(df['CO(GT)'].mean())  # 평균값 대체

In [ ]:
# K-NN 대치
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)    # default: 2
imp_knn = df.copy().values
imp_knn = imputer.fit_transform(imp_knn)

In [ ]:
# K-NN 대치 결과 변환(ndarray -> DataFrame)
imp_df = pd.DataFrame(imp_knn, index=imp_locf.index, columns=df.columns)

In [ ]:
# 결측치 처리 결과 시각화
plt.plot(df['CO(GT)'], label='actual', zorder=10)
plt.plot(imp_linear, label='linear interpolation', zorder=3)
plt.plot(imp_nocb, label='nocb', zorder=2)
plt.plot(imp_locf, label='locf', zorder=1)
plt.plot(imp_mean, label='mean substitution', zorder=4)
plt.plot(imp_df['CO(GT)'], label='k-nearest neighbor', zorder=5)
plt.legend(loc='best')
plt.show()